# Getting The Data

In [1]:
import pandas as pd
ghi_df = pd.read_csv('/content/drive/MyDrive/predicted_ghi.csv')
print(ghi_df)

    hour  predicted_ghi        date
0      0       0.000000  2025-10-09
1      1       0.000000  2025-10-09
2      2       0.000000  2025-10-09
3      3       0.000000  2025-10-09
4      4       0.000000  2025-10-09
5      5       0.000000  2025-10-09
6      6       0.000000  2025-10-09
7      7      65.597340  2025-10-09
8      8     174.463470  2025-10-09
9      9     298.220580  2025-10-09
10    10     443.440770  2025-10-09
11    11     560.139600  2025-10-09
12    12     639.329400  2025-10-09
13    13     648.836400  2025-10-09
14    14     606.947630  2025-10-09
15    15     491.622770  2025-10-09
16    16     326.187070  2025-10-09
17    17     157.321370  2025-10-09
18    18      38.067554  2025-10-09
19    19       0.000000  2025-10-09
20    20       0.000000  2025-10-09
21    21       0.000000  2025-10-09
22    22       0.000000  2025-10-09
23    23       0.000000  2025-10-09


In [2]:
import pandas as pd
load_df = pd.read_csv('/content/drive/MyDrive/predicted_load.csv')
print(load_df)

    hour  predicted_load        date
0      0       7685.0930  2025-10-09
1      1       7156.7070  2025-10-09
2      2       6732.7060  2025-10-09
3      3       6442.0140  2025-10-09
4      4       6328.8784  2025-10-09
5      5       6341.0220  2025-10-09
6      6       6510.8657  2025-10-09
7      7       6838.9023  2025-10-09
8      8       7046.9490  2025-10-09
9      9       7328.2065  2025-10-09
10    10       7572.7905  2025-10-09
11    11       7843.7420  2025-10-09
12    12       8167.2134  2025-10-09
13    13       8484.8960  2025-10-09
14    14       8830.7280  2025-10-09
15    15       9254.6880  2025-10-09
16    16       9659.0670  2025-10-09
17    17      10019.4470  2025-10-09
18    18      10153.8430  2025-10-09
19    19       9991.4900  2025-10-09
20    20       9671.2020  2025-10-09
21    21       9373.9290  2025-10-09
22    22       8919.2650  2025-10-09
23    23       8334.5970  2025-10-09


# To Find PV and Battery Sizes

In [3]:
battery_capacity_list = []
final_PV_plant_m2_list = []

def simulate_day(target_date, seasonal_load_df, seasonal_ghi_df):
    load_day = seasonal_load_df[seasonal_load_df['date'] == target_date].reset_index(drop=True)
    ghi_day = seasonal_ghi_df[seasonal_ghi_df['date'] == target_date].reset_index(drop=True)

    '''Finding the Final Solar Photovoltaic Plant in m²'''
    PV_with_bonus = 0.20 * 1.31
    PR = 0.786
    daily_load_Wh = sum(load_day.iloc[:,1]) * 1_000_000
    daily_GHI_Wh_m2 = sum(ghi_day.iloc[:,1])

    final_PV_plant_m2 = daily_load_Wh / (daily_GHI_Wh_m2 * PV_with_bonus * PR)
    final_PV_plant_m2_list.append(final_PV_plant_m2)

    print(f'Solar PV Plant Size {final_PV_plant_m2:.2f} m²\n')

    '''Hourly Solar Photovoltaic Output & Surplus/Shortage Calculations'''
    total_days_shortage = 0
    total_days_surplus = 0

    battery_state = 0  # Start at neutral (or start at full capacity if preferred)
    battery_states = []

    for hour in range(24):
        load_for_the_hour_Wh = (load_day.iloc[hour,1]) * 1000000
        hour_GHI_Wh_m2 = (ghi_day.iloc[hour,1])

        if hour_GHI_Wh_m2 > 0:
            hour_PV_output_MWh = (hour_GHI_Wh_m2 * PV_with_bonus * PR * final_PV_plant_m2) / 1000000
            diff = hour_PV_output_MWh - (load_for_the_hour_Wh / 1000000)

            if diff >= 0:
                total_days_surplus += diff
            else:
                total_days_shortage += abs(diff)

            print(
                f"{hour}: Solar PV output: {hour_PV_output_MWh:.3f} MWh & "
                f"Actual load: {load_for_the_hour_Wh / 1000000:.3f} MWh & "
                f"{'Surplus of ' if diff >= 0 else 'Shortage of -'}{abs(diff):.3f} MWh"
            )

            battery_state += diff
            battery_states.append(battery_state)

        else:
            shortage = load_for_the_hour_Wh / 1000000
            print(f'{hour}: Shortage of -{shortage:.3f} MWh')
            total_days_shortage += shortage
            battery_state -= shortage
            battery_states.append(battery_state)

    print(f'\nSurplus: {total_days_surplus:.3f} MWh in Solar PV going to Battery Storage')
    print(f'Shortage: {total_days_shortage:.3f} MWh either accessed from Battery Storage or NGCC')
    print(f'Battery states: {[f"{x:.3f}" for x in battery_states]}')

    '''Capacity of Battery'''
    # Battery capacity should cover the range from max to min state
    battery_capacity = max(battery_states) - min(battery_states)
    battery_capacity_list.append(battery_capacity)

    print(f'Required battery capacity: {battery_capacity:.3f} MWh')

In [4]:
''' Load Data for Representative Dates'''

import pandas as pd
import os

# Path to your seasonal file
save_directory = '/content/drive/MyDrive'
seasonal_file = os.path.join(save_directory, 'seasonal_predicted_loads.csv')

# Check if the file exists and is non-empty
if os.path.exists(seasonal_file) and os.path.getsize(seasonal_file) > 0:
    try:
        seasonal_load_df = pd.read_csv(seasonal_file)
        print("✅ Seasonal data loaded successfully!")
        print(seasonal_load_df)
    except pd.errors.EmptyDataError:
        print("⚠️ Seasonal file exists but is empty.")
        seasonal_load_df = pd.DataFrame()  # create empty DataFrame
else:
    print("⚠️ Seasonal file does not exist or is empty.")
    seasonal_load_df = pd.DataFrame()  # create empty DataFrame


✅ Seasonal data loaded successfully!
     hour  predicted_load        date
0       0     6730.208000  2025-03-21
1       1     6317.309600  2025-03-21
2       2     5992.536000  2025-03-21
3       3     5779.637700  2025-03-21
4       4     5609.925300  2025-03-21
..    ...             ...         ...
283    19     8018.749023  2025-02-21
284    20     8250.614258  2025-02-21
285    21     8259.625977  2025-02-21
286    22     8042.062500  2025-02-21
287    23     7776.392090  2025-02-21

[288 rows x 3 columns]


In [5]:
''' Solar GHI Data for Representative Dates'''

import pandas as pd
import os

# Path to your seasonal file
save_directory = '/content/drive/MyDrive'
GHI_file = os.path.join(save_directory, 'seasonal_predicted_ghi.csv')

# Check if the file exists and is non-empty
if os.path.exists(GHI_file) and os.path.getsize(GHI_file) > 0:
    try:
        seasonal_ghi_df = pd.read_csv(GHI_file)
        print("✅ GHI data loaded successfully!")
        print(seasonal_ghi_df)
    except pd.errors.EmptyDataError:
        print("⚠️ GHI file exists but is empty.")
        seasonal_ghi_df = pd.DataFrame()  # create empty DataFrame
else:
    print("⚠️ GHI file does not exist or is empty.")
    seasonal_ghi_df = pd.DataFrame()  # create empty DataFrame


✅ GHI data loaded successfully!
     hour  predicted_ghi        date
0       0            0.0  2025-03-21
1       1            0.0  2025-03-21
2       2            0.0  2025-03-21
3       3            0.0  2025-03-21
4       4            0.0  2025-03-21
..    ...            ...         ...
283    19            0.0  2025-02-21
284    20            0.0  2025-02-21
285    21            0.0  2025-02-21
286    22            0.0  2025-02-21
287    23            0.0  2025-02-21

[288 rows x 3 columns]


In [6]:
simulate_day("2025-03-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-04-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-05-21", seasonal_load_df, seasonal_ghi_df)

# print("--")
# print(battery_capacity_list)
# print(final_PV_plant_m2_list)

Solar PV Plant Size 186478332.01 m²

0: Shortage of -6730.208 MWh
1: Shortage of -6317.310 MWh
2: Shortage of -5992.536 MWh
3: Shortage of -5779.638 MWh
4: Shortage of -5609.925 MWh
5: Shortage of -5574.014 MWh
6: Shortage of -5655.417 MWh
7: Solar PV output: 622.828 MWh & Actual load: 5845.335 MWh & Shortage of -5222.507 MWh
8: Solar PV output: 4844.082 MWh & Actual load: 6086.470 MWh & Shortage of -1242.388 MWh
9: Solar PV output: 8154.823 MWh & Actual load: 6405.889 MWh & Surplus of 1748.934 MWh
10: Solar PV output: 11872.923 MWh & Actual load: 6707.161 MWh & Surplus of 5165.762 MWh
11: Solar PV output: 14882.075 MWh & Actual load: 6735.973 MWh & Surplus of 8146.102 MWh
12: Solar PV output: 19930.251 MWh & Actual load: 6783.007 MWh & Surplus of 13147.244 MWh
13: Solar PV output: 21620.407 MWh & Actual load: 6813.663 MWh & Surplus of 14806.744 MWh
14: Solar PV output: 21946.117 MWh & Actual load: 6976.355 MWh & Surplus of 14969.762 MWh
15: Solar PV output: 19898.112 MWh & Actual load

In [7]:
simulate_day("2025-06-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-07-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-08-21", seasonal_load_df, seasonal_ghi_df)

# print("--")
# print(battery_capacity_list)
# print(final_PV_plant_m2_list)

Solar PV Plant Size 158689228.94 m²

0: Shortage of -10040.786 MWh
1: Shortage of -9209.105 MWh
2: Shortage of -8607.014 MWh
3: Shortage of -8225.139 MWh
4: Shortage of -7970.303 MWh
5: Shortage of -7622.427 MWh
6: Shortage of -7478.985 MWh
7: Solar PV output: 2812.428 MWh & Actual load: 7438.935 MWh & Shortage of -4626.507 MWh
8: Solar PV output: 7735.044 MWh & Actual load: 7602.401 MWh & Surplus of 132.643 MWh
9: Solar PV output: 13090.135 MWh & Actual load: 8006.682 MWh & Surplus of 5083.453 MWh
10: Solar PV output: 20157.509 MWh & Actual load: 8403.241 MWh & Surplus of 11754.268 MWh
11: Solar PV output: 24960.216 MWh & Actual load: 9075.121 MWh & Surplus of 15885.094 MWh
12: Solar PV output: 28078.278 MWh & Actual load: 9823.237 MWh & Surplus of 18255.041 MWh
13: Solar PV output: 28892.244 MWh & Actual load: 10528.911 MWh & Surplus of 18363.333 MWh
14: Solar PV output: 28911.521 MWh & Actual load: 11088.818 MWh & Surplus of 17822.703 MWh
15: Solar PV output: 26697.257 MWh & Actual 

In [8]:
simulate_day("2025-09-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-10-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-11-21", seasonal_load_df, seasonal_ghi_df)

# print("--")
# print(battery_capacity_list)
# print(final_PV_plant_m2_list)

Solar PV Plant Size 189546653.38 m²

0: Shortage of -8827.731 MWh
1: Shortage of -8146.844 MWh
2: Shortage of -7746.161 MWh
3: Shortage of -7347.417 MWh
4: Shortage of -7165.840 MWh
5: Shortage of -7210.750 MWh
6: Shortage of -7434.976 MWh
7: Solar PV output: 878.316 MWh & Actual load: 7739.969 MWh & Shortage of -6861.652 MWh
8: Solar PV output: 5741.924 MWh & Actual load: 7886.763 MWh & Shortage of -2144.839 MWh
9: Solar PV output: 12562.551 MWh & Actual load: 8241.268 MWh & Surplus of 4321.283 MWh
10: Solar PV output: 18340.790 MWh & Actual load: 8642.458 MWh & Surplus of 9698.332 MWh
11: Solar PV output: 27501.977 MWh & Actual load: 9059.108 MWh & Surplus of 18442.869 MWh
12: Solar PV output: 31626.294 MWh & Actual load: 9587.666 MWh & Surplus of 22038.628 MWh
13: Solar PV output: 31383.000 MWh & Actual load: 10072.087 MWh & Surplus of 21310.913 MWh
14: Solar PV output: 29104.692 MWh & Actual load: 10523.706 MWh & Surplus of 18580.986 MWh
15: Solar PV output: 26020.066 MWh & Actual 

In [9]:
simulate_day("2025-12-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-01-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-02-21", seasonal_load_df, seasonal_ghi_df)

# print("--")
# print(battery_capacity_list)
# print(final_PV_plant_m2_list)

Solar PV Plant Size 318363810.02 m²

0: Shortage of -6590.680 MWh
1: Shortage of -6419.684 MWh
2: Shortage of -6270.017 MWh
3: Shortage of -6161.588 MWh
4: Shortage of -6253.072 MWh
5: Shortage of -6415.909 MWh
6: Shortage of -6880.918 MWh
7: Solar PV output: 792.758 MWh & Actual load: 7534.108 MWh & Shortage of -6741.350 MWh
8: Solar PV output: 7143.825 MWh & Actual load: 7992.439 MWh & Shortage of -848.614 MWh
9: Solar PV output: 13524.272 MWh & Actual load: 7997.808 MWh & Surplus of 5526.465 MWh
10: Solar PV output: 19118.565 MWh & Actual load: 7715.731 MWh & Surplus of 11402.833 MWh
11: Solar PV output: 24459.511 MWh & Actual load: 7430.483 MWh & Surplus of 17029.028 MWh
12: Solar PV output: 31635.022 MWh & Actual load: 7292.448 MWh & Surplus of 24342.574 MWh
13: Solar PV output: 31449.426 MWh & Actual load: 7249.138 MWh & Surplus of 24200.289 MWh
14: Solar PV output: 22922.805 MWh & Actual load: 7160.491 MWh & Surplus of 15762.314 MWh
15: Solar PV output: 13302.056 MWh & Actual lo

In [10]:
final_battery_size = sum(battery_capacity_list) / len(battery_capacity_list)
final_PV_plant_size = sum(final_PV_plant_m2_list) / len(final_PV_plant_m2_list)

print(f"Final Averaged Battery Capacity: {final_battery_size}")
print(f"Final Averaged PV Plant Size: {final_PV_plant_size}")

Final Averaged Battery Capacity: 109444.99001391667
Final Averaged PV Plant Size: 214271139.90200898


# Calculating TODAY'S Information

In [11]:
'''Today's Hourly Solar PV Output & Surplus/Shortage Calculations'''

amt_in_battery = 0
fossil_used = 0
solar_wasted = 0
solar_produced = 0

for hour in range(24):
    load_for_the_hour_Wh = (load_df.iloc[hour, 1]) * 1000000
    hour_GHI_Wh_m2 = (ghi_df.iloc[hour, 1])

    PV_with_bonus = 0.20 * 1.31
    PR = 0.786

    if hour_GHI_Wh_m2 > 0:
        hour_PV_output_MWh = (hour_GHI_Wh_m2 * PV_with_bonus * PR * final_PV_plant_size) / 1000000
        diff = hour_PV_output_MWh - (load_for_the_hour_Wh / 1000000)
        solar_produced += hour_PV_output_MWh

        print(
            f"{hour}: The Solar PV output: {hour_PV_output_MWh} MWh, "
            f"Actual load: {load_for_the_hour_Wh / 1000000} MWh, "
            f"{'Surplus of ' if diff >= 0 else 'Shortage of -'}{abs(diff)} MWh"
        )

        if diff >= 0:  # surplus → store in battery
            available_space = final_battery_size - amt_in_battery
            charge = min(diff, available_space)
            amt_in_battery += charge
            solar_wasted += max(0, diff - charge)  # if battery full, waste rest

        else:  # shortage → draw from battery
            needed = abs(diff)
            discharge = min(needed, amt_in_battery)
            amt_in_battery -= discharge
            fossil_used += max(0, needed - discharge)  # if battery empty, use fossil
    else:
        shortage = load_for_the_hour_Wh / 1000000
        print(f"{hour}: Shortage of -{shortage} MWh")

        discharge = min(shortage, amt_in_battery)
        amt_in_battery -= discharge
        fossil_used += max(0, shortage - discharge)

print()
print(f"{amt_in_battery} MWh remaining in the Battery Storage")
print(f"{solar_wasted} MWh of solar energy wasted (battery full)")
print(f"{fossil_used} MWh supplied by fossil fuels (battery empty)")


0: Shortage of -7685.093 MWh
1: Shortage of -7156.707 MWh
2: Shortage of -6732.706 MWh
3: Shortage of -6442.014 MWh
4: Shortage of -6328.8784 MWh
5: Shortage of -6341.022 MWh
6: Shortage of -6510.8657 MWh
7: The Solar PV output: 2894.5012822224576 MWh, Actual load: 6838.9023 MWh, Shortage of -3944.401017777542 MWh
8: The Solar PV output: 7698.250228072955 MWh, Actual load: 7046.949 MWh, Surplus of 651.3012280729554 MWh
9: The Solar PV output: 13159.067901154602 MWh, Actual load: 7328.2065 MWh, Surplus of 5830.8614011546015 MWh
10: The Solar PV output: 19566.950082956315 MWh, Actual load: 7572.7905 MWh, Surplus of 11994.159582956316 MWh
11: The Solar PV output: 24716.319143788056 MWh, Actual load: 7843.742 MWh, Surplus of 16872.577143788054 MWh
12: The Solar PV output: 28210.59158896556 MWh, Actual load: 8167.2134 MWh, Surplus of 20043.37818896556 MWh
13: The Solar PV output: 28630.09066758809 MWh, Actual load: 8484.896 MWh, Surplus of 20145.19466758809 MWh
14: The Solar PV output: 2678

In [19]:
'''Calculation if everything was done by fossil fuels'''

total_load = load_df['predicted_load'].sum()
total_Co2_emissions_for_all_NGCC = total_load * 0.499
print(f'{total_Co2_emissions_for_all_NGCC} metric tons of CO₂ will be emitted with 100% NGCC')

'''Calculate actual fossil fuel emissions'''

actual_Co2_emissions = (fossil_used * 0.499) + (solar_produced * 0.1238)
print(f'{actual_Co2_emissions} metric tons of CO₂ will actually be emitted')

'''Offsetting'''

print(f'{total_Co2_emissions_for_all_NGCC-actual_Co2_emissions} metric tons of CO₂ are NOT being emitted!!')
percentage_decrease = ((total_Co2_emissions_for_all_NGCC - actual_Co2_emissions) / total_Co2_emissions_for_all_NGCC) * 100
print(f'{percentage_decrease:.2f}% decrease of CO₂ emissions')

97149.43265819999 metric tons of CO₂ will be emitted with 100% NGCC
49829.71255111543 metric tons of CO₂ will actually be emitted
47319.72010708456 metric tons of CO₂ are NOT being emitted!!
48.71% decrease of CO₂ emissions
